# Manual contributor

This notebook extracts contribution tree data that has been exported manually
(copy-pasted) from openLCA into an Excel file, in the format shown in the
picture below.

<img src="manual_excel_example.png">

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
Baseline = pd.read_excel('..\Baseline_contributions.xlsx', 'A').dropna()

In [3]:
sub_processes = []

for i,row in Baseline.iterrows():
    if row['Process'].startswith('Baseline'):
        sub_processes.append([row])
    elif row['Process'] == 'Process':
        pass
    else:
        sub_processes[-1].append(row)

In [4]:
for i,row in Baseline.iterrows():
    if row['Process'] == 'Process':
        Baseline.drop(i, axis=0, inplace=True)

Baseline.reset_index(drop=True, inplace=True)

In [5]:
Baseline

,Contribution [%],Process,Required amount,Unit,Total result [mol H+-Eq],Direct contribution [mol H+-Eq]
0,100,Baseline - 7 Effluent treatment - RER,1000,kg,-159.763999,0
1,0.46,"market group for electricity, medium voltage |...",1616.134777,MJ,0.740521,0.740521
2,0.36,"market for hazardous waste, for underground de...",641.504971,kg,0.578281,0.578281
3,0.0858,"market for quicklime, milled, packed | quickli...",124.492587,kg,0.137044,0.137044
4,0.00431,"market for wastewater, average | wastewater, a...",3.67375,m3,0.006879,0.006879
5,0.00322,"market for wastewater, unpolluted | wastewater...",9.168921,m3,0.005146,0.005146
6,0.00147,market group for tap water | tap water | Cutof...,1460.227804,kg,0.002344,0.002344
7,101,Baseline - 6 Li recovery - RER,1000,kg,-161.234214,0
8,4.98,"market for soda ash, dense | soda ash, dense |...",1291.027446,kg,7.953329,7.953329
9,0.5,"market group for electricity, medium voltage |...",1730.082396,MJ,0.792732,0.792732


In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

pivot_df.plot.bar(stacked=True, grid=True, ax=ax, cmap=my_cmap)
plt.legend(bbox_to_anchor=(1.01, 1.0))
plt.xlabel('')
plt.xticks(rotation=60)
plt.ylabel(category)
mpl.rcParams.update({'font.size': 16})

plt.show()